In [ ]:
import cv2
import numpy as np
import os
from keras._tf_keras.keras.preprocessing import image

In [ ]:
def get_average_gray_value(image:np.array, start_row:int, end_row:int, start_col:int, end_col:int)->float:
  image_height, image_width = np.shape(image)[:2]

  value = []
  for row in range(start_row, end_row):
    for col in range(start_col, end_col):
      if row in range(image_height) and col in range(image_width):
        value.append(image[row, col])

  if (len(value) == 0):
    return 0
  
  return sum(value) / len(value)

def get_window(image:np.array, row:int, col:int, subwindow_height:int, subwindow_width:int)->np.array:
  y1 = (row - 1) * subwindow_height
  y2 = y1 + subwindow_height
  y3 = y2 + subwindow_height
  y4 = y3 + subwindow_height
  x1 = (col - 1) * subwindow_width
  x2 = x1 + subwindow_width
  x3 = x2 + subwindow_width
  x4 = x3 + subwindow_width
  area = [
      get_average_gray_value(image, y1, y2, x1, x2),
      get_average_gray_value(image, y1, y2, x2, x3),
      get_average_gray_value(image, y1, y2, x3, x4),
      get_average_gray_value(image, y2, y3, x1, x2),
      get_average_gray_value(image, y2, y3, x2, x3),
      get_average_gray_value(image, y2, y3, x3, x4),
      get_average_gray_value(image, y3, y4, x1, x2),
      get_average_gray_value(image, y3, y4, x2, x3),
      get_average_gray_value(image, y3, y4, x3, x4),
  ]
  area = np.reshape(area, (3, 3))
  return area


def get_bin_code(area:list)->list:
  center = area[1, 1]
  bin_code = [
    1 if area[0][0] >= center else 0,
    1 if area[0][1] >= center else 0,
    1 if area[0][2] >= center else 0,
    1 if area[1][2] >= center else 0,
    1 if area[2][2] >= center else 0,
    1 if area[2][1] >= center else 0,
    1 if area[2][0] >= center else 0,
    1 if area[1][0] >= center else 0,
  ]
  return bin_code

def MBLBP(image:str, window_size:int, with_cropping=False)->np.array:
  """Compute the Multi Block local binary patterns (MB-LBP) of an image.

    The Multi-Block Local Binary Pattern (MB-LBP) extends traditional LBP by analyzing multiple surrounding blocks, allowing it to capture a broader range of textural information from images. This technique is particularly effective in recognizing patterns and features across different scales and orientations, making it robust against variations in lighting and pose. MB-LBP has been widely adopted in computer vision tasks such as facial recognition, texture classification, and surveillance systems due to its enhanced descriptive and discriminative capabilities.

    Parameters
    ----------
    image : (M, N) array
        2D grayscale image.
    window_size : int
        Number of circularly symmetric neighbor set points (quantization of
        the angular space).

    Returns
    -------
    output : (M, N) array
        MB-LBP Image.
  """
  image = cv2.imread(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
  image_height, image_width = np.shape(image)[:2]
  window_height = window_size
  window_width =  window_size 
  if window_height % 3 != 0 or window_width % 3 != 0:
    raise ValueError("Invalid MBLBP Size")

  subwindow_height = window_height // 3
  subwindow_width = window_width // 3
  total_subwindow_y = image_height // subwindow_height
  total_subwindow_x = image_width // subwindow_width

  mblbp_image_height = total_subwindow_y * subwindow_height
  mblbp_image_width = total_subwindow_x * subwindow_width
  mblbp_image = np.zeros((mblbp_image_height, mblbp_image_width), np.uint8)
  for row in range(total_subwindow_y):
    for col in range(total_subwindow_x):
      area = get_window(image, row, col, subwindow_height, subwindow_width)
      bin_code = get_bin_code(area)
      mblbp_code = int("".join(map(str, bin_code))[::-1], 2)
      mblbp_image[
        row * subwindow_height : (row + 1) * subwindow_height,
        col * subwindow_width : (col + 1) * subwindow_width
      ] = mblbp_code
  
  return mblbp_image

In [ ]:
def extract_mblbp_features(paths, window_size:int):
    mblbp_features = []
    for image_path in paths: 
        mblbp = MBLBP(image=image_path, window_size=window_size)
        if (len(mblbp) > 0) :
            mblbp_histogram, _ = np.histogram(mblbp.ravel(), bins=np.arange(0, 257), range=(0, 256))
            mblbp_histogram = mblbp_histogram.astype("float")
            mblbp_features.append(mblbp_histogram)
        else:
            print(f"image are not valid. {image_path}")
    return mblbp_features

In [ ]:
def extract_mblbp_features_from_directory(directory_path, window_size, output_file_name):
    categories = {"male": 0, "female": 1}

    image_paths = []
    labels = []

    for category, label in categories.items():
        category_path = os.path.join(directory_path, category)
        for img_name in sorted(os.listdir(category_path)):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                img_path = os.path.join(category_path, img_name)    
                image_paths.append(img_path)
                labels.append(label)

    features_array = extract_mblbp_features(paths=image_paths, window_size=window_size)
    labels_array = np.array(labels)

    np.savez(f'{output_file_name}_{window_size}x{window_size}.npz', features=features_array, labels=labels_array)


In [ ]:
# scheme 1
extract_mblbp_features_from_directory(directory_path="../dataset/scheme-1", window_size=3, output_file_name="mblbp_1")
extract_mblbp_features_from_directory(directory_path="../dataset/scheme-1", window_size=6, output_file_name="mblbp_1")
extract_mblbp_features_from_directory(directory_path="../dataset/scheme-1", window_size=9, output_file_name="mblbp_1")

In [ ]:
# scheme 2
extract_mblbp_features_from_directory(directory_path="../dataset/scheme-2", window_size=3, output_file_name="mblbp_2")
extract_mblbp_features_from_directory(directory_path="../dataset/scheme-2", window_size=6, output_file_name="mblbp_2")
extract_mblbp_features_from_directory(directory_path="../dataset/scheme-2", window_size=9, output_file_name="mblbp_2")